In [73]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Simple EDA

In [74]:
pokemon = pd.read_csv("/kaggle/input/pokemon/Pokemon.csv")

In [75]:
pokemon.head()

In [76]:
pokemon.columns

In [77]:
pokemon.columns = ['id', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense', 
                   'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary']

In [78]:
pokemon.tail()

# Data Visualization

In [79]:
pokemon['Dual-Type'] = pokemon['Type 1'].fillna('') + ' ' + pokemon['Type 2'].fillna('')
pokemon['Dual-Type'] = pokemon['Dual-Type'].str.strip()

print(f"Number of types in Type 1: {len(pokemon['Type 1'].dropna().unique())}")
print(f"Number of types in Type 2: {len(pokemon['Type 2'].dropna().unique())}")
print(f"Number of types in Dual-Type: {len(pokemon['Dual-Type'].unique())}")

In [80]:
pokemon['Dual-Type']

In [81]:
count_type_1 = pokemon.groupby(by = ['Generation', 'Type 1'], dropna = True).count()[['id']]

count_type_1.reset_index(inplace = True)

count_type_1.rename(columns = {'Type 1' : 'Type'}, inplace = True)

count_type_1.set_index(['Generation', 'Type'], inplace = True)

count_type_1

In [82]:
count_type_2 = pokemon.groupby(by = ['Generation', 'Type 2'], dropna = True).count()[['id']]

count_type_2.reset_index(inplace = True)

count_type_2.rename(columns = {'Type 2' : 'Type'}, inplace = True)

count_type_2.set_index(['Generation', 'Type'], inplace = True)

count_type_2

In [83]:
count_single_type = count_type_1.add(count_type_2, axis = 'columns', fill_value = 0)

del count_type_1, count_type_2

count_single_type['id'] = count_single_type['id'].astype('int')

count_single_type.reset_index(level = [0, 1], inplace = True)

count_single_type

In [84]:
#count_single_type.loc['all'] = df.apply(lambda x : x.groupby(by = ['Type'])['id'].sum())

sum_single_type = pd.pivot_table(count_single_type, index = ['Type'], values = ['id'], aggfunc=np.sum)

sum_single_type['id'] = sum_single_type['id'].astype('int')

sum_single_type.reset_index(inplace = True)

sum_single_type['Generation'] = 'all'

single_type = pd.concat([count_single_type, sum_single_type])  

del sum_single_type

single_type.reset_index(drop = True, inplace = True)

single_type

## matplotlib

In [85]:
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.image as mpimg
import seaborn as sns

import os
import re
from collections import OrderedDict

In [86]:
figsize = 10

fig, ax = plt.subplots(figsize = (figsize, figsize))

def creat_pokeball(size, generation):
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    
    Pokeball_Crimson = '#EE1515'
    Pokeball_Shark = '#222224'
    Pokebll_Gallery = '#F0F0F0'
    
    palette_generation_1 = ['#FF7327', '#00DD00']
    palette_generation_2 = ['#B69E00', '#C0C0E1']
    palette_generation_3 = ['#AB2813', '#26649C']
    palette_generation_4 = ['#44BAE5', '#DA7D99']
    palette_generation_5 = ['#444444', '#E1E1E1']
    palette_generation_6 = ['#025DA6', '#EA1A3E']
    palette_generation_all = [Pokeball_Crimson, Pokebll_Gallery]
    palettes = [palette_generation_1, palette_generation_2, palette_generation_3,
                palette_generation_4, palette_generation_5, palette_generation_6,
                palette_generation_all]
    if generation in [1, 2, 3, 4, 5, 6]:
        pokeball_colors = palettes[generation - 1]
    elif generation == 'all':
        pokeball_colors = palettes[6]
    else:
        raise ValueError("The entered generation information is incorrect.")
        
    wedges_0, texts_0 = ax.pie(
        x = [100],
        labels = ['GEN\n' + str(generation).upper()],
        colors = [Pokebll_Gallery],
        startangle = 0,
        explode = [0] * len([100]),
        wedgeprops = {'width' : 1.15 * size},
        radius = 1.15 * size,
    )
    
    for i, wedge in enumerate(wedges_0):
        texts_0[i].set_color(Pokeball_Crimson)
        texts_0[i].set_fontsize(24 * size)
        texts_0[i].set_fontstyle('normal')
        texts_0[i].set_fontweight('roman')
        texts_0[i].set_horizontalalignment('center')
        texts_0[i].set_position((0, 0))
        texts_0[i].set_alpha(0.75)
        
    wedges_1 = ax.pie(
        x = [100],
        colors = [Pokeball_Shark],
        startangle = 0,
        explode = [0] * len([100]),
        wedgeprops = {'width': 0.8 * size},
        radius = 1.15 * size,
    )
    
    wedges_2 = ax.pie(
        x = [50, 50],
        colors = pokeball_colors,
        startangle = 0,
        explode = [0.05 * size] * len([50, 50]),
        wedgeprops = {'width': 0.85 * size, 'linewidth': 8 * size, 'edgecolor': Pokeball_Shark},
        radius = 1.2 * size,
    )
    
    wedges_3 = ax.pie(
        x = [100],
        colors = [Pokebll_Gallery],
        explode = [0] * len([100]),
        startangle = 0,
        wedgeprops = dict(width = 0.05 * size, linewidth = 16 * size, edgecolor = Pokeball_Shark),
        radius = 0.4 * size,
    )
    
    wedges_4 = ax.pie(
        x = [100],
        colors = [Pokebll_Gallery],
        explode = [0] * len([100]),
        startangle = 0,
        wedgeprops = dict(width = 0.25 * size, linewidth = 4 * size, edgecolor = Pokeball_Shark),
        radius = 0.25 * size,
    )

creat_pokeball(figsize / 10, 'all')

plt.show()

In [87]:
def starter_classifier():
    n_generation_1 = ['001', '004', '007']
    n_generation_2 = ['152', '155', '158']
    n_generation_3 = ['252', '255', '258']
    n_generation_4 = ['387', '390', '393']
    n_generation_5 = ['495', '498', '501']
    n_generation_6 = ['650', '653', '656']
    n_generation_All = ['025', '133']
    
    image_generation_1 = {}
    image_generation_2 = {}
    image_generation_3 = {}
    image_generation_4 = {}
    image_generation_5 = {}
    image_generation_6 = {}
    image_generation_All = {}
    
    for dirname, _, filenames in os.walk('../input/pokemon-starters-by-generation/'):
        for filename in filenames:
            n_pokedex = re.split('\.|_', filename)[1]
            if n_pokedex in n_generation_1:
                image_generation_1.update({n_pokedex: os.path.join(dirname, filename)})
            elif n_pokedex in n_generation_2:
                image_generation_2.update({n_pokedex: os.path.join(dirname, filename)})
            elif n_pokedex in n_generation_3:
                image_generation_3.update({n_pokedex: os.path.join(dirname, filename)})
            elif n_pokedex in n_generation_4:
                image_generation_4.update({n_pokedex: os.path.join(dirname, filename)})
            elif n_pokedex in n_generation_5:
                image_generation_5.update({n_pokedex: os.path.join(dirname, filename)})
            elif n_pokedex in n_generation_6:
                image_generation_6.update({n_pokedex: os.path.join(dirname, filename)})
            else:
                image_generation_All.update({n_pokedex: os.path.join(dirname, filename)})
                
    return image_generation_1, image_generation_2, image_generation_3, image_generation_4, image_generation_5, image_generation_6, image_generation_All

In [88]:
figsize = 10

fig, ax = plt.subplots(figsize = (figsize, figsize))

def add_starters(size, generation):
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    creat_pokeball(size, generation)
    ns_pokedex = []
    image_paths = []
    if generation in [1, 2, 3, 4, 5, 6]:
        dict_starters = starter_classifier()[generation - 1]
        for key in sorted(dict_starters):
            ns_pokedex.append(key)
            image_paths.append(dict_starters[key])
        image_1 = mpimg.imread(image_paths[0])
        image_2 = mpimg.imread(image_paths[1])
        image_3 = mpimg.imread(image_paths[2])
        imagebox_1 = OffsetImage(image_1, zoom = .45 * size)
        imagebox_2 = OffsetImage(image_2, zoom = .45 * size)
        imagebox_3 = OffsetImage(image_3, zoom = .45 * size)
        annotationbox_1 = AnnotationBbox(imagebox_1, (-.52 * size, -.3 * size), frameon = False)
        annotationbox_2 = AnnotationBbox(imagebox_2, (0 * size, .6 * size), frameon = False)
        annotationbox_3 = AnnotationBbox(imagebox_3, (.52 * size, -.3 * size), frameon = False)
        ax.add_artist(annotationbox_1)
        ax.add_artist(annotationbox_2)
        ax.add_artist(annotationbox_3)
    elif generation == 'all':
        dict_starters = starter_classifier()[6]
        for key in sorted(dict_starters):
            ns_pokedex.append(key)
            image_paths.append(dict_starters[key])
        image_1 = mpimg.imread(image_paths[0])
        image_2 = mpimg.imread(image_paths[1])
        imagebox_1 = OffsetImage(image_1, zoom = .4 * size)
        imagebox_2 = OffsetImage(image_2, zoom = .4 * size)
        annotationbox_1 = AnnotationBbox(imagebox_1, (-.52 * size, -.3 * size), frameon = False)
        annotationbox_2 = AnnotationBbox(imagebox_2, (.52 * size, -.3 * size), frameon = False)
        ax.add_artist(annotationbox_1)
        ax.add_artist(annotationbox_2)
    else:
        raise ValueError("The entered generation information is incorrect.")

add_starters(figsize / 10, 'all')

plt.show()

In [89]:
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [91]:
from pathlib import Path
directory = 'pokeballs'
parent_dir = '/kaggle/working'
path = os.path.join(parent_dir, directory)
from pathlib import Path
Path(path).mkdir(parents = True, exist_ok = True)
my_path = os.path.abspath(path)
my_path

In [92]:
from matplotlib.artist import Artist 

figsize = 10

fig, ax = plt.subplots(figsize = (figsize, figsize))

for i in [1, 2, 3, 4, 5, 6, 'all']:
    fig = add_starters(figsize / 10, i)
    save_name = my_path + "/pokeball_" + str(i) + ".png"
    plt.savefig(fname = save_name, transparent = True)
    plt.cla()
plt.clf()
plt.close()

In [ ]:
figsize = 10

fig, ax = plt.subplots(figsize = (figsize, figsize))

def bar_chart(size, generation, df):
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    
    Pokebll_Gallery = '#F0F0F0'
    
    df_generation = df.loc[df['Generation'] == generation]
    
    proportion_all = zip(sorted(single_type.loc[single_type['Generation'] == 'all']['id']) /
                     single_type.loc[single_type['Generation'] == 'all']['id'].sum(),
                     sorted(single_type.loc[single_type['Generation'] == 'all']['Type']))
    wedges_width = {}
    for i , j in proportion_all:
        wedges_width.update({j : i})
    sorted_types = sorted(df_generation['Type'])
    sorted_wedges_width = OrderedDict([(i, wedges_width[i]) for i in sorted_types])
    
    colors = {
        'Bug': '#A8B820',
        'Dark': '#705848',
        'Dragon': '#7038F8',
        'Electric': '#F8D030',
        'Fairy': '#F0B6BC',
        'Fighting': '#C03028',
        'Fire': '#F08030',
        'Flying': '#A890F0',
        'Ghost': '#705898',
        'Grass': '#78C850',
        'Ground': '#E0C068',
        'Ice': '#98D8D8',
        'Normal': '#A8A878',
        'Poison': '#A040A0',
        'Psychic': '#F85888',
        'Rock': '#B8A038',
        'Steel': '#B8B8D0',
        'Water': '#6890F0'
    }
    
    edgecolors = {
        'Bug': '#D8E030',
        'Dark': '#A8A878',
        'Dragon': '#B8A0F8',
        'Electric': '#F8F878',
        'Fairy': '#F5CAD1',
        'Fighting': '#F08030',
        'Fire': '#F8D030',
        'Flying': '#C8C0F8',
        'Ghost': '#A890F0',
        'Grass': '#C0F860',
        'Ground': '#F8F878',
        'Ice': '#D0F8E8',
        'Normal': '#D8D8D0',
        'Poison': '#D880B8',
        'Psychic': '#F8C0B0',
        'Rock': '#E0C068',
        'Steel': '#D8D8C0',
        'Water': '#98D8D8'
    }
    
    wedges_1, texts_1, autotexts_1 = ax.pie(
        x = sorted(df_generation['id']),
        labels = sorted(df_generation['Type']),
        colors = df_generation['Type'].apply(lambda x: colors[x]),
        autopct = '%.2f%%',
        startangle = 0,
        labeldistance = 0,
        pctdistance = .57,
        wedgeprops = dict(linewidth = 2 * size, edgecolor = Pokebll_Gallery, alpha = 0.75),
        radius = 2.5 * size,
    )
    
    for i, wedge in enumerate(wedges_1):
        wedges_1[i].set_edgecolor(edgecolors.get(wedge.get_label()))
    
    for i in texts_1:
        i.set(visible = False)
    
    for wedge, i in zip(wedges_1, sorted_wedges_width):
        wedge.set_radius(1.2 * size + 10 * sorted_wedges_width.get(i) * size)
    
    count = 0
    for t in autotexts_1:
        t.set_fontsize((sorted_wedges_width.get(i) + count) * size * 100)
        count += 0.005
    
    [t.set_color(Pokebll_Gallery) for t in autotexts_1]

    add_starters(size, generation)

bar_chart(figsize / 10, 'all', single_type)

plt.show()

In [ ]:
figsize = 10

fig = plt.figure(figsize = (3 * figsize, 3 * figsize))

ax = fig.add_gridspec(3, 3)
axes = []
axes.append(fig.add_subplot(ax[0, 0]))
axes.append(fig.add_subplot(ax[0, 1]))
axes.append(fig.add_subplot(ax[0, 2]))
axes.append(fig.add_subplot(ax[2, 0]))
axes.append(fig.add_subplot(ax[2, 1]))
axes.append(fig.add_subplot(ax[2, 2]))
axes.append(fig.add_subplot(ax[1, 0: ]))

Pokeball_Crimson = '#EE1515'
Pokeball_Shark = '#222224'
Pokebll_Gallery = '#F0F0F0'
    
colors = {
        'Bug': '#A8B820',
        'Dark': '#705848',
        'Dragon': '#7038F8',
        'Electric': '#F8D030',
        'Fairy': '#F0B6BC',
        'Fighting': '#C03028',
        'Fire': '#F08030',
        'Flying': '#A890F0',
        'Ghost': '#705898',
        'Grass': '#78C850',
        'Ground': '#E0C068',
        'Ice': '#98D8D8',
        'Normal': '#A8A878',
        'Poison': '#A040A0',
        'Psychic': '#F85888',
        'Rock': '#B8A038',
        'Steel': '#B8B8D0',
        'Water': '#6890F0'
    }

for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)

for i in [1, 2, 3, 4, 5, 6, 'all']:
    if i == 'all':
        ax = axes[6]
        sub = bar_chart(figsize / 10, i, single_type)
        legend = ax.legend(title = 'Pokemon Types',
                    title_fontsize = 'xx-large',
                    bbox_to_anchor = (0, 0, 2, .9),
                    labels = "Type " + single_type.loc[single_type['Generation'] == i]['Type'].astype('str'),
                    labelcolor = single_type.loc[single_type['Generation'] == i]['Type'].apply(lambda x: colors[x]),
                    fontsize = 'x-large',
                    facecolor = Pokebll_Gallery,
                    edgecolor = Pokeball_Shark)
        plt.setp(legend.get_title(),
                 color = Pokeball_Crimson,
                 fontstyle = 'normal',
                 fontfamily = 'monospace')
    else:
        ax = axes[i - 1]
        sub = bar_chart(figsize / 18, i, single_type)

title = fig.suptitle("Distribution of Pokemon Types in Different Generations",
                     fontweight = 'black',
                     fontstretch = 'expanded',
                     fontstyle = 'normal',
                     fontfamily = 'monospace',
                     fontsize = 30,
                     y = .9,
                     color = Pokeball_Crimson)

fig.savefig('pokemon_type_analysis_matplotlib.png', transparent = True)

plt.show()

In [ ]:
from io import BytesIO
from PIL import Image


import base64

from skimage import io, filters, measure, morphology, img_as_ubyte
from matplotlib import cm



import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

import base64
from io import BytesIO
from skimage import io, filters, measure, morphology, img_as_ubyte
from matplotlib import cm

In [ ]:
from matplotlib.artist import Artist 

figsize = 10

fig, ax = plt.subplots(figsize = (figsize, figsize))

for i in [1, 2, 3, 4, 5, 6, 'all']:
    fig = add_starters(figsize / 10, i)
    save_name = my_path + "/pokeball_" + str(i) + ".png"
    plt.savefig(fname = save_name, transparent = True)
    plt.cla()
plt.clf()
plt.close()

In [ ]:
images = [io.imread('./pokeballs/pokeball_all.png'), io.imread('./pokeballs/pokeball_1.png')]


def to_png(arr):
    out = BytesIO()
    im = Image.fromarray(arr)
    im.save(out, format = 'png')
    return out.getvalue()


#def b64_image_files(images, colormap = 'magma'):
#    cmap = cm.get_cmap(colormap)
#    urls = []
#    for im in tqdm(images):
#        #png = to_png(img_as_ubyte(cmap(im)))
#        png = to_png(im)
#        url = 'data:image/png;base64,' + base64.b64encode(png).decode('utf-8')
#        urls.append(url)
#    return urls

def b64_image_files(images, colormap = 'magma'):
    cmap = cm.get_cmap(colormap)
    urls = []
    for im in images:
        png = to_png(img_as_ubyte(cmap(im)))
        url = 'data:image/png;base64,' + base64.b64encode(png).decode('utf-8')
        urls.append(url)
    return urls


colormap = 'magma'

cmap = cm.get_cmap(colormap)

urls = []

for im in images:
    png = to_png(im)
    url = 'data:image/png;base64,' + base64.b64encode(png).decode('utf-8')
    urls.append(url)

from bokeh.plotting import figure, show, output_file

p = figure(x_range=(0,1), y_range=(0,1))


p.image_url(url=urls[0], x=0, y=1, w=0.8, h=0.6)
show(p)

In [ ]:
os.remove('./image.html')

In [ ]:
img = io.imread('./pokeballs/pokeball_all.png')

img

In [ ]:
images = [io.imread('./pokeballs/pokeball_all.png'), io.imread('./pokeballs/pokeball_1.png')]

images